#### 給定隊伍等級與裁判隊伍自動分組與更新ref_conflict表

In [5]:
import pandas as pd
import random

# === 讀取 Excel 資料 ===
excel_data = pd.read_excel("referees.xlsx", sheet_name=None)
level_df = excel_data["level"]
ref_team_df = excel_data["ref_team"]

# === 整理資料 ===
# Get teams and their levels
level_df = level_df.rename(columns={"team_name": "team", "level": "level"})
team_levels = level_df.set_index("team")["level"].to_dict()

# Randomly assign teams to groups
teams_by_level = {level: [] for level in range(1, 5)}  # Levels 1 to 4
for team, level in team_levels.items():
    teams_by_level[level].append(team)

# Check team availability for groups A to H (levels 1 to 4)
required_teams_level_1_to_4 = 8  # 8 groups (A to H)
for level in range(1, 5):
    if len(teams_by_level[level]) < required_teams_level_1_to_4:
        raise ValueError(f"Not enough teams at level {level}: need {required_teams_level_1_to_4}, but only {len(teams_by_level[level])} available.")

grouped_teams = {}
# Groups A to H (one from each level 1 to 4)
for group in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    selected_teams = [random.choice(teams_by_level[level]) for level in range(1, 5)]
    for team in selected_teams:
        teams_by_level[team_levels[team]].remove(team)
    grouped_teams[group] = selected_teams

# Group I (3 teams, prefer one from levels 1, 2, 3 if possible)
remaining_teams = []
for level in range(1, 5):
    remaining_teams.extend(teams_by_level[level])

if len(remaining_teams) < 3:
    raise ValueError(f"Not enough remaining teams for group I: need 3, but only {len(remaining_teams)} available.")

# Try to select one team from each of levels 1, 2, 3
selected_teams = []
available_levels = [level for level in range(1, 4) if teams_by_level[level]]
if len(available_levels) >= 3:
    for level in available_levels[:3]:
        if teams_by_level[level]:
            team = random.choice(teams_by_level[level])
            selected_teams.append(team)
            teams_by_level[level].remove(team)
            remaining_teams.remove(team)
else:
    # Fallback: randomly select 3 teams from remaining teams
    selected_teams = random.sample(remaining_teams, 3)
    for team in selected_teams:
        if team in remaining_teams:
            remaining_teams.remove(team)
            level = team_levels[team]
            if team in teams_by_level[level]:
                teams_by_level[level].remove(team)

grouped_teams['I'] = selected_teams

# Group J (all remaining teams, regardless of level)
remaining_teams = []
for level in range(1, 5):
    remaining_teams.extend(teams_by_level[level])

if remaining_teams:
    grouped_teams['J'] = remaining_teams
    if len(remaining_teams) < 2:
        print(f"⚠️ 警告：Group J 只有 {len(remaining_teams)} 支隊伍，無法形成比賽。")
else:
    print("⚠️ 警告：沒有剩餘隊伍可分配到 Group J。")

# Create DataFrame for groupings
grouping_data = []
for group, teams in grouped_teams.items():
    for team in teams:
        grouping_data.append({"Group": group, "Team": team, "Level": team_levels[team]})

grouping_df = pd.DataFrame(grouping_data)

# === Generate Referee Conflict Table ===
# Get unique referees from ref_team
referees = ref_team_df["name"].unique()

# Initialize conflict dictionary
ref_conflicts = {}
for ref in referees:
    ref_conflicts[ref] = {}

# Fill conflict table: 0 if referee has conflict with any team in group, 1 otherwise
for ref in referees:
    conflicted_teams = set(ref_team_df[ref_team_df["name"] == ref]["dept"])
    for group, teams in grouped_teams.items():
        # Check if referee has conflict with any team in the group
        has_conflict = any(team in conflicted_teams for team in teams)
        ref_conflicts[ref][group] = 0 if has_conflict else 1

# Convert to DataFrame
ref_conflict_df = pd.DataFrame(ref_conflicts).T
ref_conflict_df.index.name = "Referee"

# === 輸出到 Excel ===
with pd.ExcelWriter("group_generate.xlsx", engine="openpyxl") as writer:
    grouping_df.to_excel(writer, sheet_name="Groupings", index=False)
    ref_conflict_df.to_excel(writer, sheet_name="Referee Conflicts")

print("✅ 分組和裁判衝突表已生成！")
print("📤 已輸出：group_generate.xlsx（包含 'Groupings' 和 'Referee Conflicts' 兩頁）")

✅ 分組和裁判衝突表已生成！
📤 已輸出：group_generate.xlsx（包含 'Groupings' 和 'Referee Conflicts' 兩頁）


### 排賽程 using Gurobi

In [15]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from itertools import combinations

# === 讀取 Excel 資料 ===
# Load groupings and referee conflicts
group_data = pd.read_excel("group_generate.xlsx", sheet_name=None)
grouping_df = group_data["Groupings"]
ref_conflict_df = group_data["Referee Conflicts"]

# Load referee and team availability
avail_data = pd.read_excel("臺大盃女排資料表.xlsx", sheet_name=None)
ref_df = avail_data["ref_available"]
teams_df = avail_data["team_available"]

# === 整理資料 ===
# Process referee availability (keep original values: 0, 0.5, 1)
ref_df = ref_df.set_index("name").T
referees = list(ref_df.columns)
days = list(ref_df.index.astype(int))
fields = list(range(4))

# Process referee conflicts
ref_conflict_df = ref_conflict_df.set_index("Referee")
referee_conflicts = ref_conflict_df.to_dict("index")
for ref in referee_conflicts:
    referee_conflicts[ref] = {group: int(value) for group, value in referee_conflicts[ref].items()}

# Process team availability
teams_df = teams_df.rename(columns={"Unnamed: 0": "team"})
team_week_avail = teams_df.set_index("team").to_dict("index")

# Process groupings
grouped_teams = {}
for group, group_df in grouping_df.groupby("Group"):
    grouped_teams[group] = list(group_df["Team"])

# Get list of all teams
teams = [team for group in grouped_teams.values() for team in group]

# Map days to weekdays
weekdays = ["mon", "tue", "wed", "thur", "fri"]
day_to_weekday = {d: weekdays[d % 5] for d in days if d % 5 < 5}
valid_days = [d for d in days if d % 5 < 5]

# === 建立比賽清單（照分組，單循環）===
matches = []
for group, team_list in grouped_teams.items():
    if len(team_list) < 2:
        print(f"⚠️ 警告：Group {group} 只有 {len(team_list)} 支隊伍，無法形成比賽，將不會出現在賽程表中。")
        continue
    for t1, t2 in combinations(team_list, 2):
        matches.append((group, t1, t2))

# === 建立 Gurobi 模型 ===
m = Model("volleyball_schedule")
m.setParam('OutputFlag', 0)

# Decision variables
x = {}
for group, t1, t2 in matches:
    for d in valid_days:
        wd = day_to_weekday[d]
        if team_week_avail[t1][wd] and team_week_avail[t2][wd]:
            for f in fields:
                for r in referees:
                    if ref_df.loc[d, r] >= 0.5:
                        if referee_conflicts[r][group] == 1:
                            x[(group, t1, t2, d, f, r)] = m.addVar(vtype=GRB.BINARY)

makespan = m.addVar(vtype=GRB.INTEGER)

# Variables to track the number of games each referee officiates
referee_games = {}
for r in referees:
    referee_games[r] = m.addVar(vtype=GRB.INTEGER, name=f"referee_games_{r}")

# Constraint: calculate the number of games each referee officiates
for r in referees:
    m.addConstr(
        referee_games[r] == quicksum(x.get((g, t1, t2, d, f, rr), 0)
                                    for (g, t1, t2, d, f, rr) in x if rr == r),
        name=f"referee_games_count_{r}"
    )

# Variables to track deviation from the average number of games
total_matches = len(matches)
avg_games = total_matches / len(referees) if referees else 0
deviation_plus = {r: m.addVar(vtype=GRB.CONTINUOUS, name=f"dev_plus_{r}") for r in referees}
deviation_minus = {r: m.addVar(vtype=GRB.CONTINUOUS, name=f"dev_minus_{r}") for r in referees}

# Constraints for deviation
for r in referees:
    m.addConstr(referee_games[r] == avg_games + deviation_plus[r] - deviation_minus[r])

# === 限制條件 ===
# 一場比賽只能排一次
for group, t1, t2 in matches:
    m.addConstr(quicksum(x.get((group, t1, t2, d, f, r), 0)
                         for d in valid_days for f in fields for r in referees
                         if (group, t1, t2, d, f, r) in x) == 1)

# 同一場地每天至多一場
for d in valid_days:
    for f in fields:
        m.addConstr(quicksum(x.get((g, t1, t2, dd, ff, r), 0)
                             for (g, t1, t2, dd, ff, r) in x
                             if dd == d and ff == f) <= 1)

# 同一裁判每天至多執法一場
for d in valid_days:
    for r in referees:
        m.addConstr(quicksum(x.get((g, t1, t2, dd, f, rr), 0)
                             for (g, t1, t2, dd, f, rr) in x
                             if dd == d and rr == r) <= 1)

# 每隊每天至多一場比賽
for t in teams:
    for d in valid_days:
        m.addConstr(quicksum(x.get((g, t1, t2, dd, f, r), 0)
                             for (g, t1, t2, dd, f, r) in x
                             if dd == d and (t1 == t or t2 == t)) <= 1)

# makespan 控制比賽結束日期
for key in x:
    group, t1, t2, d, f, r = key
    m.addConstr(x[key] * d <= makespan)

# === 目標函數 ===
penalty = quicksum(0.1 * x[key] for key in x if ref_df.loc[key[3], key[5]] == 0.5)
balance_penalty = quicksum(deviation_plus[r] + deviation_minus[r] for r in referees)
m.setObjective(makespan + penalty + 0.01 * balance_penalty, GRB.MINIMIZE)

# === 執行求解 ===
m.optimize()

# === 輸出每日賽程表 ===
if m.status == GRB.OPTIMAL:
    # Calculate objective value components
    penalty_val = sum(0.1 * var.X for key, var in x.items() if ref_df.loc[key[3], key[5]] == 0.5)
    balance_penalty_val = sum(deviation_plus[r].X + deviation_minus[r].X for r in referees)
    objective_value = makespan.X + penalty_val + 0.01 * balance_penalty_val

    result = [(group, t1, t2, d, f, r)
              for (group, t1, t2, d, f, r), var in x.items() if var.X > 0.5]
    df = pd.DataFrame(result, columns=["Group", "Team1", "Team2", "Day", "Field", "Referee"])

    # 建立每日賽程表
    df["Match"] = df["Team1"] + " vs " + df["Team2"]
    schedule_df = df[["Day", "Field", "Match", "Referee"]].sort_values(by=["Day", "Field"])

    # Calculate referee workload
    referee_counts = {r: 0 for r in referees}
    for _, row in df.iterrows():
        referee_counts[row["Referee"]] += 1
    referee_counts_df = pd.DataFrame.from_dict(referee_counts, orient='index', columns=["Games Officiated"])
    referee_counts_df.index.name = "Referee"

    # 輸出到 Excel
    with pd.ExcelWriter("volleyball_schedule_optimal.xlsx", engine="openpyxl") as writer:
        schedule_df.to_excel(writer, sheet_name="Schedule", index=False)
        grouping_df.to_excel(writer, sheet_name="Groupings", index=False)
        referee_counts_df.to_excel(writer, sheet_name="Referee Counts")
    
    print("✅ 排程成功！最晚比賽日：", int(makespan.X))
    print(f"📊 Objective value = {objective_value:.4f} "
          f"(makespan={makespan.X:.0f}, penalty={penalty_val:.4f}, balance_penalty={balance_penalty_val:.4f})")
    print("📤 已輸出：volleyball_schedule_optimal.xlsx（包含 'Schedule'、'Groupings' 和 'Referee Counts' 三頁）")
else:
    print("❌ 無可行排程")

✅ 排程成功！最晚比賽日： 14
📊 Objective value = 14.1400 (makespan=14, penalty=0.0000, balance_penalty=14.0000)
📤 已輸出：volleyball_schedule_optimal.xlsx（包含 'Schedule'、'Groupings' 和 'Referee Counts' 三頁）


### 與Heuristic比較

In [14]:
import pandas as pd
import os
from itertools import combinations
import random

# Create 'schedules' folder if it doesn't exist
if not os.path.exists("schedules"):
    os.makedirs("schedules")

# Weekday mapping (5-day cycle: Day 1 = Monday, Day 5 = Friday, Day 6 = Monday)
WEEKDAYS = {1: "mon", 2: "tue", 3: "wed", 4: "thur", 0: "fri"}

def get_weekday(day):
    """Map day number to weekday name (5-day cycle)."""
    return WEEKDAYS[day % 5]

def get_week_number(day):
    """Determine the week number (1-based) for a given day (7-day weeks)."""
    return (day - 1) // 7 + 1

def generate_matches(group_df):
    """Generate all possible matches per group."""
    matches = []
    groups = group_df['Group'].unique()
    for group in groups:
        teams = group_df[group_df['Group'] == group]['Team'].tolist()
        if len(teams) < 2:
            print(f"⚠️ Warning: Group {group} has only {len(teams)} teams, no matches generated.")
            continue
        for team1, team2 in combinations(teams, 2):
            matches.append({
                'group': group,
                'team1': team1,
                'team2': team2,
                'match_str': f"{team1} vs {team2}",
                'scheduled': False
            })
    return matches

def find_available_referee(ref_df, ref_conflicts, group, day, referee_day_assignments, referee_counts):
    """Find an available referee for a match on a given day, checking conflicts."""
    available_refs = []
    
    # Validate group exists in ref_conflicts
    if group not in ref_conflicts.columns:
        return None, 0
    
    # Try referees with availability 1
    for ref in ref_df.columns:
        if ref == 'name':
            continue
        if (ref_conflicts.loc[ref, group] == 1 and
            ref_df.loc[day, ref] == 1 and
            ref not in referee_day_assignments.get(day, [])):
            available_refs.append((ref, referee_counts.get(ref, 0), 1))
    
    # Try referees with availability 0.5
    for ref in ref_df.columns:
        if ref == 'name':
            continue
        if (ref_conflicts.loc[ref, group] == 1 and
            ref_df.loc[day, ref] == 0.5 and
            ref not in referee_day_assignments.get(day, [])):
            available_refs.append((ref, referee_counts.get(ref, 0), 0.5))
    
    # Select referee with fewest games officiated
    if available_refs:
        available_refs.sort(key=lambda x: (x[1], -x[2]))  # Sort by games, then prefer avail=1
        return available_refs[0][0], available_refs[0][2]
    return None, 0

def schedule_matches():
    """Schedule matches using heuristic and save to output file."""
    # Load input data
    try:
        group_data = pd.read_excel("group_generate.xlsx", sheet_name=None)
        grouping_df = group_data["Groupings"]
        ref_conflict_df = group_data["Referee Conflicts"]
        
        avail_data = pd.read_excel("臺大盃女排資料表.xlsx", sheet_name=None)
        ref_df = avail_data["ref_available"]
        teams_df = avail_data["team_available"]
    except Exception as e:
        print(f"Error loading input files: {e}")
        return -1
    
    # Process data
    # Referee availability
    ref_df = ref_df.set_index("name").T
    ref_df.index = ref_df.index.astype(int)
    referees = list(ref_df.columns)
    
    # Referee conflicts
    ref_conflict_df = ref_conflict_df.set_index("Referee")
    
    # Team availability
    teams_df = teams_df.rename(columns={"Unnamed: 0": "team"})
    team_week_avail = teams_df.set_index("team")
    
    # Generate matches
    matches = generate_matches(grouping_df)
    groups = sorted(grouping_df['Group'].unique())
    
    # Initialize tracking
    schedule = []
    team_day_assignments = {}  # {day: [teams]}
    team_weekly_counts = {}  # {(week, team): count}
    referee_counts = {}  # {referee: count}
    referee_day_assignments = {}  # {day: [referees]}
    courts = [0, 1, 2, 3]
    last_match_day = 0
    half_availability_count = 0
    
    # Valid days (exclude days 31-35)
    valid_days = [d for d in range(1, 71) if d not in range(31, 36)]
    
    # Schedule matches
    for day in valid_days:
        weekday = get_weekday(day)
        week = get_week_number(day)
        courts_used = []
        matches_scheduled = 0
        
        random.shuffle(groups)
        for group in groups:
            if matches_scheduled >= 4:
                break
            
            group_matches = [m for m in matches if m['group'] == group and not m['scheduled']]
            random.shuffle(group_matches)
            
            for match in group_matches:
                if matches_scheduled >= 4:
                    break
                
                team1, team2 = match['team1'], match['team2']
                
                # Check team availability
                try:
                    if (team_week_avail.loc[team1, weekday] != 1 or
                        team_week_avail.loc[team2, weekday] != 1):
                        continue
                except KeyError:
                    continue
                
                # Check daily limit
                day_teams = team_day_assignments.get(day, [])
                if team1 in day_teams or team2 in day_teams:
                    continue
                
                # Check weekly limit (optional, can be relaxed)
                team1_week_count = team_weekly_counts.get((week, team1), 0)
                team2_week_count = team_weekly_counts.get((week, team2), 0)
                if team1_week_count >= 2 or team2_week_count >= 2:
                    continue
                
                # Find referee
                referee, ref_avail = find_available_referee(
                    ref_df, ref_conflict_df, group, day,
                    referee_day_assignments, referee_counts
                )
                if not referee:
                    continue
                
                # Assign court
                available_courts = [c for c in courts if c not in courts_used]
                if not available_courts:
                    continue
                court = available_courts[0]
                
                # Track half-availability penalty
                if ref_avail == 0.5:
                    half_availability_count += 1
                
                # Schedule match
                schedule.append({
                    'Day': day,
                    'Group': group,
                    'Field': court,
                    'Match': match['match_str'],
                    'Referee': referee
                })
                match['scheduled'] = True
                courts_used.append(court)
                matches_scheduled += 1
                last_match_day = max(last_match_day, day)
                
                # Update tracking
                team_day_assignments.setdefault(day, []).extend([team1, team2])
                team_weekly_counts[(week, team1)] = team1_week_count + 1
                team_weekly_counts[(week, team2)] = team2_week_count + 1
                referee_counts[referee] = referee_counts.get(referee, 0) + 1
                referee_day_assignments.setdefault(day, []).append(referee)
        
        # Clean up weekly counts
        team_weekly_counts = {k: v for k, v in team_weekly_counts.items() if k[0] == week}
    
    # Check unscheduled matches
    unscheduled = [m['match_str'] for m in matches if not m['scheduled']]
    if unscheduled:
        print(f"{len(unscheduled)} matches could not be scheduled: {unscheduled}")
    else:
        print("All matches scheduled successfully.")
    
    # Print last match day
    if last_match_day > 0:
        print(f"Last match scheduled on day {last_match_day} ({get_weekday(last_match_day)})")
    else:
        print("No matches were scheduled.")
    
    # Calculate objective function
    makespan = last_match_day
    penalty = 0.1 * half_availability_count
    total_matches = len(matches) - len(unscheduled)
    avg_games = total_matches / len(referees) if referees else 0
    balance_penalty = sum(
        abs(referee_counts.get(r, 0) - avg_games)
        for r in referees
    )
    objective_value = makespan + penalty + 0.01 * balance_penalty
    print(f"Objective value = {objective_value:.4f} "
          f"(makespan={makespan}, penalty={penalty:.4f}, balance_penalty={balance_penalty:.4f})")
    
    # Prepare output
    schedule_df = pd.DataFrame(schedule)
    ref_counts_df = pd.DataFrame({
        'Referee': list(referee_counts.keys()),
        'Games Officiated': list(referee_counts.values())
    })
    # Include all referees
    for ref in referees:
        if ref not in referee_counts:
            ref_counts_df = pd.concat([
                ref_counts_df,
                pd.DataFrame({'Referee': [ref], 'Games Officiated': [0]})
            ], ignore_index=True)
    
    # Save output
    output_file = "volleyball_schedule_heuristic.xlsx"
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        schedule_df.to_excel(writer, sheet_name="Schedule", index=False)
        grouping_df.to_excel(writer, sheet_name="Groupings", index=False)
        ref_counts_df.to_excel(writer, sheet_name="Referee Counts", index=False)
    
    print(f"📤 Output saved to {output_file}")
    return len(unscheduled)

# Run scheduling
unscheduled_count = schedule_matches()
print(f"Generated volleyball_schedule_heuristic.xlsx with {unscheduled_count} unscheduled matches.")

All matches scheduled successfully.
Last match scheduled on day 18 (wed)
Objective value = 18.8960 (makespan=18, penalty=0.8000, balance_penalty=9.6000)
📤 Output saved to volleyball_schedule_heuristic.xlsx
Generated volleyball_schedule_heuristic.xlsx with 0 unscheduled matches.
